In [4]:
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Function to load and preprocess the dataset
def load_dataset(data_dir):
    images = []
    annotations = []

    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            with open(os.path.join(data_dir, filename)) as json_file:
                data = json.load(json_file)
                # Assuming your images are stored in the same directory with same name but .jpg extension
                image_path = os.path.join(data_dir, os.path.splitext(filename)[0])
                if os.path.exists(image_path):
                    images.append(cv2.imread(image_path))
                    # Extracting coordinates for nose tip and mouth
                    nose_x = data["nose_tip"]["coordinates"]["x"]
                    nose_y = data["nose_tip"]["coordinates"]["y"]
                    mouth_x = data["mouth"]["coordinates"]["x"]
                    mouth_y = data["mouth"]["coordinates"]["y"]
                    annotations.append([nose_x, nose_y, mouth_x, mouth_y])

    return images, annotations

# Load your dataset
data_dir = "/home/selvatharrun/Documents/GitHub/cephalometric-analysis/train3"
images, annotations = load_dataset(data_dir)
# Preprocess your data
# You need to write code to preprocess your images and annotations

# Split your dataset
X_train, X_val, y_train, y_val = train_test_split(images, annotations, test_size=0.2, random_state=42)

def create_custom_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(4, activation='linear'))  # Assuming output shape is [nose_x, nose_y, mouth_x, mouth_y]
    return model


print(y_train)
print(".............................")
print(y_val)   
model = create_custom_model(X_train.shape[1:])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train your model
history = model.fit(np.array(X_train), np.array(y_train), 
                    validation_data=(np.array(X_val), np.array(y_val)), 
                    epochs=1, batch_size=0.2)

# Evaluate your model
# You need to write code to evaluate your model on the test set
# Monitor training
# Optionally, you can plot the training and validation loss curves
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

[[388, 101, 105, 305], [13, 8, 175, 216], [866, 1316, 870, 1286]]
.............................
[[941, 673, 946, 667]]


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import cv2

# Load an image
image = cv2.imread('example_image.jpg')

# Get the shape of the image
height, width, channels = image.shape

# Print the input_shape
input_shape = (height, width, channels)
print("Input shape of the image:", input_shape)


In [12]:
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# Function to load and preprocess the dataset
def load_dataset(data_dir):
    images = []
    annotations = []

    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            with open(os.path.join(data_dir, filename)) as json_file:
                data = json.load(json_file)
                # Assuming your images are stored in the same directory with same name but .jpg extension
                image_path = os.path.join(data_dir, os.path.splitext(filename)[0])
                if os.path.exists(image_path):
                    images.append(cv2.imread(image_path))
                    # Extracting coordinates for nose tip and mouth
                    nose_x = data["nose_tip"]["coordinates"]["x"]
                    nose_y = data["nose_tip"]["coordinates"]["y"]
                    mouth_x = data["mouth"]["coordinates"]["x"]
                    mouth_y = data["mouth"]["coordinates"]["y"]
                    annotations.append([nose_x, nose_y, mouth_x, mouth_y])

    return images, annotations

# Load your dataset
data_dir = "/home/selvatharrun/Documents/GitHub/cephalometric-analysis/train3"
images, annotations = load_dataset(data_dir)
print(images)
print(annotations)

[array([[[245, 244, 246],
        [244, 244, 247],
        [245, 244, 246],
        ...,
        [187, 187, 187],
        [187, 187, 187],
        [187, 187, 187]],

       [[245, 245, 246],
        [246, 245, 248],
        [246, 245, 247],
        ...,
        [187, 187, 187],
        [187, 187, 187],
        [187, 187, 187]],

       [[246, 245, 247],
        [245, 245, 248],
        [245, 244, 246],
        ...,
        [187, 187, 187],
        [187, 187, 187],
        [187, 187, 187]],

       ...,

       [[ 58,  34,  82],
        [ 58,  33,  82],
        [ 59,  35,  84],
        ...,
        [ 19,  27,  42],
        [ 23,  31,  46],
        [ 16,  22,  38]],

       [[ 58,  34,  82],
        [ 58,  33,  82],
        [ 59,  34,  83],
        ...,
        [ 17,  24,  38],
        [ 24,  32,  46],
        [ 23,  29,  45]],

       [[ 59,  35,  83],
        [ 59,  34,  83],
        [ 59,  34,  84],
        ...,
        [ 10,  16,  31],
        [ 11,  17,  33],
        [ 15,  22,  39]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import json
from io import BytesIO
import requests

# Define the model
def create_custom_model(input_shape):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(4, activation='linear'))  # Output shape is [nose_x, nose_y, mouth_x, mouth_y]
    return model

# Preprocess the data
X_train, y_train = [], []
target_size = (64, 64)  # Adjust this based on your model input shape


X_train = np.array(images)
y_train = np.array(annotations)

# Create the model
model = create_custom_model(input_shape=X_train.shape[1:])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)



In [ ]:
#prediction
input_image = r"/Users/harsunpranavrs/Downloads/hehe.jpg"
predictions = model.predict(input_image)

# Extract nose and mouth coordinates from predictions
nose_x, nose_y, mouth_x, mouth_y = predictions.flatten()

print("Predicted Nose Coordinates:", nose_x, nose_y)
print("Predicted Mouth Coordinates:", mouth_x, mouth_y)